Data cleansing for exploration/model building

Pandas dataframes are used here. For large datasets, technologies like spark or R are recommended.

In [141]:
import pandas as pd
import json

From exploring the datasets, it is found that there are certain fields which are json objects represented as strings. These fields are first formatted into actual json fields so that the data available in them can be processed

It is recommended to create a metadata repository that will compare the metadata like count and presence or absence of fields during subsequent runs of the data cleansing program. The program should compare the current input against this repository and alert the data team about changes (new fields, datatype changes, etc)

In [142]:
movies = pd.read_csv("tmdb_5000_movies.csv")

json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
for column in json_columns:
    movies[column] = movies[column].apply(json.loads)

movieCredits = pd.read_csv('tmdb_5000_credits.csv')
json_columns = ['cast','crew']
for column in json_columns:
    movieCredits[column] = movieCredits[column].apply(json.loads)

Helper functions to extract data from json objects and format them/ add them to base dataset

In [143]:
def getDirector(data):
    directors = [crewMember['name'] for crewMember in data if crewMember['job'] == 'Director']
    if directors:
        return directors[0]
    else:
        return pd.np.nan
    
def concatNames(vec):
    res = ""
    for rec in vec:
        res+=rec['name'] + "|"
    return res[:-1]

def getActor(jsonData,index):
    res = ""
    for k in jsonData:
        if k['order'] == index:
            res = k['name']
            return res
    return pd.np.nan

In [144]:
movies['release_date'] = pd.to_datetime(movies['release_date']).apply(lambda x: x.date())
movies['director_name'] = movieCredits['crew'].apply(getDirector)
movies['release_year'] = pd.to_datetime(movies['release_date']).apply(lambda x:x.year)
movies['genres'] = movies['genres'].apply(concatNames)
movies['keywords'] = movies.keywords.apply(concatNames)
movies['production_companies'] = movies.production_companies.apply(concatNames)
movies['spoken_languages'] = movies.spoken_languages.apply(concatNames)
movies['lead_actor_1'] = movieCredits.cast.apply(getActor,index=0)
movies['lead_actor_2'] = movieCredits.cast.apply(getActor,index=1)
movies['lead_actor_3'] = movieCredits.cast.apply(getActor,index=2)
movies['director_movie_count'] = movies.director_name.map(movies.director_name.value_counts())

Check for duplicate movies and remove dupes, if any

In [146]:
if len(movies) > len(movies.drop_duplicates(subset=['title','release_date'])):
    print "duplicates exist"
    dupes = movies[movies.title.map(movies.title.value_counts()>1)]
    movies = movies.drop_duplicates(subset=['title','release_date'])


In [147]:
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,tagline,title,vote_average,vote_count,director_name,release_year,lead_actor_1,lead_actor_2,lead_actor_3,director_movie_count
0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners|Twentieth Century Fox ...,...,Enter the World of Pandora.,Avatar,7.2,11800,James Cameron,2009.0,Sam Worthington,Zoe Saldana,Sigourney Weaver,7.0
1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures|Jerry Bruckheimer Films|S...,...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Gore Verbinski,2007.0,Johnny Depp,Orlando Bloom,Keira Knightley,7.0
2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures|Danjaq|B24,...,A Plan No One Escapes,Spectre,6.3,4466,Sam Mendes,2015.0,Daniel Craig,Christoph Waltz,Léa Seydoux,7.0
3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures|Warner Bros.|DC Entertainme...,...,The Legend Ends,The Dark Knight Rises,7.6,9106,Christopher Nolan,2012.0,Christian Bale,Michael Caine,Gary Oldman,8.0
4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,...,"Lost in our world, found in another.",John Carter,6.1,2124,Andrew Stanton,2012.0,Taylor Kitsch,Lynn Collins,Samantha Morton,4.0
5,258000000,Fantasy|Action|Adventure,http://www.sonypictures.com/movies/spider-man3/,559,dual identity|amnesia|sandstorm|love of one's ...,en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,Columbia Pictures|Laura Ziskin Productions|Mar...,...,The battle within.,Spider-Man 3,5.9,3576,Sam Raimi,2007.0,Tobey Maguire,Kirsten Dunst,James Franco,11.0
6,260000000,Animation|Family,http://disney.go.com/disneypictures/tangled/,38757,hostage|magic|horse|fairy tale|musical|princes...,en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,Walt Disney Pictures|Walt Disney Animation Stu...,...,They're taking adventure to new lengths.,Tangled,7.4,3330,Byron Howard,2010.0,Zachary Levi,Mandy Moore,Donna Murphy,1.0
7,280000000,Action|Adventure|Science Fiction,http://marvel.com/movies/movie/193/avengers_ag...,99861,marvel comic|sequel|superhero|based on comic b...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,Marvel Studios|Prime Focus|Revolution Sun Studios,...,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767,Joss Whedon,2015.0,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,3.0
8,250000000,Adventure|Fantasy|Family,http://harrypotter.warnerbros.com/harrypottera...,767,witch|magic|broom|school of witchcraft|wizardr...,en,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",98.885637,Warner Bros.|Heyday Films,...,Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,7.4,5293,David Yates,2009.0,Daniel Radcliffe,Rupert Grint,Emma Watson,3.0
9,250000000,Action|Adventure|Fantasy,http://www.batmanvsupermandawnofjustice.com/,209112,dc comics|vigilante|superhero|based on comic b...,en,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,155.790452,DC Co